### Here , we are importing all the required libraries

In [27]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import time
import pytz
import os
import sys
import requests
from urllib.parse import urlparse, parse_qs
from fyers_apiv3 import fyersModel
import webbrowser

### Then we are generating a unique authentication code for our new login in our account

In [28]:
redirect_uri= "https://trade.fyers.in/api-login/redirect-uri/index.html"       ## redircet_uri you entered while creating APP.

client_id =  "put_your_unique_client_id_here"                                   ## Client_id here refers to APP_ID of the created app
secret_key = "put_your_unique_secret_id_here"                                  ## app_secret key which you got after creating the app 

grant_type = "authorization_code"                                              ## The grant_type always has to be "authorization_code"
response_type = "code"                                                         ## The response_type always has to be "code"
state = "sample"                                                               

# Connect to the sessionModel object here with the required input parameters
appSession = fyersModel.SessionModel(client_id = client_id, redirect_uri = redirect_uri,response_type=response_type,state=state,secret_key=secret_key,grant_type=grant_type)

# Make  a request to generate_authcode object this will return a login url 
# which you need to open in your browser 
# from where you can get the generated auth_code 
generateTokenUrl = appSession.generate_authcode()

print((generateTokenUrl))  

webbrowser.open(generateTokenUrl,new=1)

# run the code firstly upto this after you generate the auth_code comment the above code and start executing the below code

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=1DCDP9SIPV-100&redirect_uri=https%3A%2F%2Ftrade.fyers.in%2Fapi-login%2Fredirect-uri%2Findex.html&response_type=code&state=sample


True

### After succesfull login , kindly copy the generated auth_code over here and make the request to generate the accessToken 

In [29]:
auth_code = input("Kindly enter the copied auth_code here - ")

### Now we are logging in our FYERS trading account

In [30]:
appSession.set_token(auth_code)
response = appSession.generate_token()

try: 
    access_token = response["access_token"]
except Exception as e:
    print(e,response) 


fyers = fyersModel.FyersModel(token=access_token,is_async=False,client_id=client_id,log_path="")

print("We have successfully logged into your FYERS account having details ")

# print( fyers.get_profile() )

We have successfully logged into your FYERS account having details 


### All the required functions are executed below -

In [31]:
IST = pytz.timezone('Asia/Kolkata')

def get_expiry():
    global expiry, df_symbols, next_expiry, df_symbols_next_exp, lot_size
    fyers_fo = "https://public.fyers.in/sym_details/NSE_FO.csv"
    df_symbols = pd.read_csv(fyers_fo, index_col=False, header=None)
    df_symbols = df_symbols[df_symbols[13] =='BANKNIFTY' ]
    df_symbols.reset_index(drop=True, inplace=True)
    df_symbols[8] = pd.to_datetime(df_symbols[8],unit='s').dt.date
    exps = sorted(df_symbols[8].unique())
    expiry = exps[0]
    next_expiry = exps[1]
    print()
    print(f"Current date and timing is - {datetime.today().now()}")
    print(f"Current Week Expiry is on - {expiry}")
    print(f"Next Week Expiry is on - {next_expiry}")
    
    df_symbols_next_exp = df_symbols[df_symbols[8]==next_expiry]
    df_symbols = df_symbols[df_symbols[8]==expiry]
    lot_size = df_symbols.loc[0,3]

def get_symbol(strike, ce=True,next_expiry=False):
    try:
        if next_expiry:
            if ce == True:
                return(df_symbols_next_exp[df_symbols_next_exp[1].str.contains(strike+' CE')].iloc[0,9])
            else:
                return(df_symbols_next_exp[df_symbols_next_exp[1].str.contains(strike+' PE')].iloc[0,9])
        else:
            if ce == True:
                return(df_symbols[df_symbols[1].str.contains(strike+' CE')].iloc[0,9])
            else:
                return(df_symbols[df_symbols[1].str.contains(strike+' PE')].iloc[0,9])
    except Exception as e:
        print(f"Error while fetching symbol {strike} and CE {ce}- {e}")
        
def get_ltp(symbol):
    quote = fyers.quotes({"symbols":symbol})
    print(quote)                                  # this gives following error when our token expires
    if quote['d'][0]['n'] == symbol:              # {'s': 'error', 'code': -8, 'message': 'Your token has expired. Please generate a token'}
        return(quote['d'][0]['v']['lp'])          # solution = call the check() code line       

def get_all_symbols(next_expiry=False):
    global ce_list, pe_list
    ce_list = []
    pe_list = []
    i=20   # no of strikes above and below the spot price
    
    # bn_atm = round(get_ltp("NSE:BANKNIFTY24JULFUT") , -2)

    while i != 0:
        ce_list.append(get_symbol(str(bn_atm + i*100), ce=True,next_expiry= next_expiry))
        pe_list.append(get_symbol(str(bn_atm - i*100), ce=False, next_expiry=next_expiry))
        i = i - 1

    return ce_list + pe_list


get_expiry()


Current date and timing is - 2024-07-01 11:47:41.525957
Current Week Expiry is on - 2024-07-03
Next Week Expiry is on - 2024-07-10


### This is the main portion of our code

In [32]:
def get_ltp(symbol):
    quote = fyers.quotes({"symbols":symbol})
    print(quote)                              
    if quote['d'][0]['n'] == symbol:          
        return(quote['d'][0]['v']['lp'])      

In [33]:
bn_atm = int(round(get_ltp("NSE:BANKNIFTY24JULFUT") , -2))

{'message': '', 'code': 200, 'd': [{'n': 'NSE:BANKNIFTY24JULFUT', 'v': {'ask': 52730, 'bid': 52722.5, 'chp': 0.36, 'ch': 191.3, 'description': 'NSE:BANKNIFTY24JULFUT', 'exchange': 'NSE', 'fyToken': '101124073135165', 'high_price': 52779.9, 'low_price': 52301, 'lp': 52721, 'open_price': 52524, 'original_name': 'NSE:BANKNIFTY24JULFUT', 'prev_close_price': 52529.7, 'short_name': 'BANKNIFTY24JULFUT', 'spread': 7.5, 'symbol': 'NSE:BANKNIFTY24JULFUT', 'tt': '1719792000', 'volume': 1048410}, 's': 'ok'}], 's': 'ok'}


In [34]:
print(f"We are initialising the algo at {datetime.today().now()}")

# yf_scrips = [ "MCX:NATURALGAS23NOVFUT" , "MCX:CRUDEOIL23NOVFUT" ]
# yf_scrips = [ "NSE:BANKNIFTY2JULFUT" ] 
# yf_scrips = ["NSE:NIFTYBANK-INDEX"]
yf_scrips = get_all_symbols()

print()
print("Scripts choosen by you for trading are :-")
print(yf_scrips)


We are initialising the algo at 2024-07-01 11:47:41.838687

Scripts choosen by you for trading are :-
['NSE:BANKNIFTY2470354700CE', 'NSE:BANKNIFTY2470354600CE', 'NSE:BANKNIFTY2470354500CE', 'NSE:BANKNIFTY2470354400CE', 'NSE:BANKNIFTY2470354300CE', 'NSE:BANKNIFTY2470354200CE', 'NSE:BANKNIFTY2470354100CE', 'NSE:BANKNIFTY2470354000CE', 'NSE:BANKNIFTY2470353900CE', 'NSE:BANKNIFTY2470353800CE', 'NSE:BANKNIFTY2470353700CE', 'NSE:BANKNIFTY2470353600CE', 'NSE:BANKNIFTY2470353500CE', 'NSE:BANKNIFTY2470353400CE', 'NSE:BANKNIFTY2470353300CE', 'NSE:BANKNIFTY2470353200CE', 'NSE:BANKNIFTY2470353100CE', 'NSE:BANKNIFTY2470353000CE', 'NSE:BANKNIFTY2470352900CE', 'NSE:BANKNIFTY2470352800CE', 'NSE:BANKNIFTY2470350700PE', 'NSE:BANKNIFTY2470350800PE', 'NSE:BANKNIFTY2470350900PE', 'NSE:BANKNIFTY2470351000PE', 'NSE:BANKNIFTY2470351100PE', 'NSE:BANKNIFTY2470351200PE', 'NSE:BANKNIFTY2470351300PE', 'NSE:BANKNIFTY2470351400PE', 'NSE:BANKNIFTY2470351500PE', 'NSE:BANKNIFTY2470351600PE', 'NSE:BANKNIFTY2470351700PE'

In [35]:
def fetch_current_time():
    return list(map(int, datetime.today().now(IST).strftime("%H %M %S").split()))


def historical_bydate(symbol,sd,ed):
        data = {"symbol":symbol,"resolution":"1","date_format":"1","range_from":str(sd),"range_to":str(ed),"cont_flag":"1"}
        # print(data)
        nx = fyers.history(data)
        # print(nx)
        nx1 = nx["candles"]
        cols = ['datetime','open','high','low','close','volume']
        df = pd.DataFrame.from_dict(nx1)
        df.columns = cols
        df['datetime'] = pd.to_datetime(df['datetime'],unit = "s")
        df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
        df['datetime'] = df['datetime'].dt.tz_localize(None)
        df = df.set_index('datetime')
        return df
    
def place_order(symb, qnty, order_side, lmt, stop_price, stop_loss, take_profit):
    fyers.place_order(data = {"symbol" : symb,"qty" : qnty,"type" : 4,"side" : order_side,"productType" : "INTRADAY","limitPrice" : lmt,"stopPrice" : stop_price,"disclosedQty" : 0,"validity" : "DAY","offlineOrder" : "False","stopLoss" : stop_loss,"takeProfit" : take_profit})    

In [36]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/rIHcv2fdom8?si=4EmVUv_n8BtEJS4b&amp;start=32" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen>

In [37]:
# The inside bar strategy implemented below is explained in above embedded video.
# Kindly check the above video for clear understanding of whole logic of this strategy.

def inside_bar(margin,symb,hi,lo,cl,op):

    # margin is our total balance available for trading
    # symb is the name of element which we want to trade
    # hi,lo,cl,op are lists containing corresponding - high , low , close  , open - prices

    # for FYERS                                    # -> via body length , we are checking green or red candle
    len_ic = hi[-2] - lo[-2]                       # length of body of inside candle    
    body_ic = cl[-2] - op[-2]                      # total length of inside candle
    body_length = cl[-3] - op[-3]                  # length of body of mother candle
    candle_length = hi[-3] - lo[-3]                # total length of mother candle      

    period = list(map(int, datetime.today().now(IST).strftime("%H %M %S").split()))

    print("I am scanning for inside bars in "+ symb + " at " + str(period) , end = "\n" )
    
    if body_ic*body_length >= 0:
        return 0
    
    elif (hi[-3] - lo[-3] / hi[-3] > 0.8):
        
        if (lo[-2] > lo[-3]) and (hi[-2] <= hi[-3]):

            if body_length > 0:
                print(f"Bar having high - {hi[-2]} and low - {lo[-2]} is found inside mother bar having high - {hi[-3]} and low - {lo[-3]}")
                print(f"---->   Going long for {symb} at price {hi[-2]} " + str(period))
                qnty = 1
                lt = 0.05 + hi[-2]

                order_data = place_order(symb,qnty,1,lt,hi[-2],len_ic,2*len_ic)       
                return 1

            else:
                print(f"---->   Going short for {symb} at price {lo[-2]} " + str(period))
                print(f"Bar having high - {hi[-2]} and low - {lo[-2]} is found inside mother bar having high - {hi[-3]} and low - {lo[-3]}")
                qnty = 1
                lt = lo[-2] - 0.05

                order_data = place_order(symb,qnty,-2,lt,lo[-2],len_ic,2*len_ic)                 
                return -2
    else:
        return 0
                       
      

margin = 300000                     #Total Capital for trading in INR
interval = 1                        # Candle Interval (1,5,15,30,60)  ( Recommended is 15 for buying )

In [39]:
while(1):
    
    current_time = fetch_current_time()
        
    if current_time[1]%interval == 0 :    

        startdate = datetime.today().date() - timedelta(days=5)
        enddate = datetime.today().date()
        his_data = historical_bydate( yf_scrips[0] , startdate , enddate )
        
        latest_fetched_bar_time = fetch_current_time()

        if (latest_fetched_bar_time[1] - 2 )   <  current_time[1]  <  (latest_fetched_bar_time[1] + 2 ) : 

            order_ids = []
            pos = fyers.positions()
            pos_list = []

            try:
                pos_list = list(pd.DataFrame(pos["data"]["netPositions"])['symbol'])
            except:
                pass

            for i in range(len(yf_scrips)):
    
                if yf_scrips[i] in pos_list:
                    continue

                try :
                    data = historical_bydate( yf_scrips[i] , startdate , enddate)
                except:
                    print(f"Error in fetching historical data for {yf_scrips[i]}")
                    continue

                hi = list(data['high'])
                lo = list(data['low'])
                op = list(data['open'])
                cl = list(data['close'])
                
                result = inside_bar( margin, yf_scrips[i] , hi, lo, cl, op )        
                
            

            current_time = fetch_current_time()
            rem_time = 60 - current_time[2]
           
            print("Let's wait for " + str( (abs(rem_time)) ) + " seconds for next data point") 
            
            while rem_time > 0 :
                print("." ,  end = " ")
                rem_time = rem_time - 1
                time.sleep(1)
                
    print()
    print()    
    print()

I am scanning for inside bars in NSE:BANKNIFTY2470354700CE at [12, 48, 13]
I am scanning for inside bars in NSE:BANKNIFTY2470354600CE at [12, 48, 14]
I am scanning for inside bars in NSE:BANKNIFTY2470354500CE at [12, 48, 14]
I am scanning for inside bars in NSE:BANKNIFTY2470354400CE at [12, 48, 14]
I am scanning for inside bars in NSE:BANKNIFTY2470354300CE at [12, 48, 15]
I am scanning for inside bars in NSE:BANKNIFTY2470354200CE at [12, 48, 15]
I am scanning for inside bars in NSE:BANKNIFTY2470354100CE at [12, 48, 15]
I am scanning for inside bars in NSE:BANKNIFTY2470354000CE at [12, 48, 16]
I am scanning for inside bars in NSE:BANKNIFTY2470353900CE at [12, 48, 16]
I am scanning for inside bars in NSE:BANKNIFTY2470353800CE at [12, 48, 16]
I am scanning for inside bars in NSE:BANKNIFTY2470353700CE at [12, 48, 17]
I am scanning for inside bars in NSE:BANKNIFTY2470353600CE at [12, 48, 17]
I am scanning for inside bars in NSE:BANKNIFTY2470353500CE at [12, 48, 17]
I am scanning for inside 

KeyboardInterrupt: 